<a href="https://colab.research.google.com/github/sunam128/2-Point-Classification/blob/main/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity #matrice pairwise

In [3]:
#load the files
ratings=pd.read_csv("ratings.csv")
movies=pd.read_csv("movies.csv")

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [9]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [10]:
movies.isnull().sum()

,0
movieId,0
title,0
genres,0


In [11]:
#similarity matrix
#pivot_table means summary
user_movie=ratings.pivot_table(
    index="userId",
    columns="movieId",
    values="rating"
)

In [12]:
#user fails to put rating
movie_vectors=user_movie.fillna(0).T  #.T swaps the rows and the columns for easier readability

In [13]:
#Calculate the similarity
movie_sim=pd.DataFrame(
    cosine_similarity(movie_vectors),
    index=movie_vectors.index,
    columns=movie_vectors.index
)

In [14]:
from scipy.stats.distributions import recipinvgauss_gen
#main function
def recommend_movie():
  #ask for the movie name
  movie_name=input("Enter the movie name: ")
  #find the matches
  matches=movies[movies["title"].str.contains(movie_name,case=False,na=False)]
  #movie is not found
  if matches.empty:
    print("Movie not found")
    return
  #pick the first matching movie
  movie_id=matches.iloc[0]["movieId"]
  movie_title=matches.iloc[0]["title"]
  #organize the similar movies
  similar_movies=movie_sim[movie_id].sort_values(ascending=False)[1:11]
  #more movie details i.e. genre
  recs=movies[movies["movieId"].isin(similar_movies.index)]
  #print the recommended movies
  result=recs[["title","genres"]].copy()
  return result
recommend_movie()

Enter the movie name: Matilda


,title,genres
521,Homeward Bound II: Lost in San Francisco (1996),Adventure|Children
551,James and the Giant Peach (1996),Adventure|Animation|Children|Fantasy|Musical
559,Space Jam (1996),Adventure|Animation|Children|Comedy|Fantasy|Sc...
773,Homeward Bound: The Incredible Journey (1993),Adventure|Children|Drama
1052,101 Dalmatians (1996),Adventure|Children|Comedy
1177,Hercules (1997),Adventure|Animation|Children|Comedy|Musical
1191,George of the Jungle (1997),Children|Comedy
1543,"Jungle Book, The (1967)",Animation|Children|Comedy|Musical
1550,Peter Pan (1953),Animation|Children|Fantasy|Musical
2976,How the Grinch Stole Christmas (a.k.a. The Gri...,Children|Comedy|Fantasy
